In [1]:
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/noisy-eval/kopitiam.json_0_to_1000-test.json
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/noisy-eval/twitter.json_0_to_1000-test.json

In [2]:
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/bjn_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/eng_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/ind_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/jav_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zsm_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zho_Hans.dev

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-base-malaysian-cased')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/translation-nanot5-tiny-malaysian-cased')

2023-10-06 00:08:22.099075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 00:08:22.186173: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-06 00:08:22.634466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-06 00:08:22.634503: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [5]:
_ = model.cuda()

In [7]:
s = 'Hai, ada yang bisa saya bantu?'
input_ids = tokenizer.encode(f'terjemah ke Melayu: {s}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
outputs = model.generate(input_ids, max_length = 100)
print(tokenizer.batch_decode(outputs, skip_special_tokens = True))

['Hai, boleh saya tolong?']


In [15]:
files = {
    'bjn': 'bjn_Latn.dev',
    'en': 'eng_Latn.dev',
    'ind': 'ind_Latn.dev',
    'jav': 'jav_Latn.dev',
    'ms': 'zsm_Latn.dev',
    'mandarin': 'zho_Hans.dev',
}

In [17]:
data = {}
for k, v in files.items():
    with open(v) as fopen:
        t = fopen.read().split('\n')
    
    t = [s for s in t if len(s)]
    data[k] = t
    print(k, len(t))

bjn 997
en 997
ind 997
jav 997
ms 997
mandarin 997


In [28]:
pairs = [
    ['en', 'ms'],
    ['ms', 'en'],
    ['ind', 'ms'],
    ['jav', 'ms'],
    ['mandarin', 'ms'],
    ['mandarin', 'en']
]

In [29]:
from tqdm import tqdm
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [43]:
mapping = {
    'ms': 'Melayu',
    'en': 'Inggeris',
    'manglish': 'Manglish',
    'pasar ms': 'pasar Melayu',
}

In [39]:
results = {}
for left, right in pairs:
    predicted = []
    l = mapping[right]
    for s in tqdm(data[left]):
        input_ids = tokenizer.encode(f'terjemah ke {l}: {s}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
        outputs = model.generate(input_ids, max_length = 1024)
        predicted.append(tokenizer.batch_decode(outputs, skip_special_tokens = True)[0])
    
    score = chrf.corpus_score(predicted, [data[right][:len(predicted)]])
    results[f'{left}-{right}'] = score

100%|█████████████████████████████████████████| 997/997 [01:17<00:00, 12.92it/s]


In [40]:
results

{'en-ms': chrF2++ = 63.61,
 'ms-en': chrF2++ = 59.55,
 'ind-ms': chrF2++ = 56.38,
 'jav-ms': chrF2++ = 47.68,
 'mandarin-ms': chrF2++ = 39.78,
 'mandarin-en': chrF2++ = 36.61}

In [41]:
import json
import pandas as pd

with open('kopitiam.json_0_to_1000-test.json') as fopen:
    data = json.load(fopen)
    
manglish = []
for d in data:
    try:
        r = json.loads(d[1])
        d = {
            'manglish': d[0],
            'ms': r['malay'],
            'en': r['british_english'],
        }
        if isinstance(d['manglish'], str) and isinstance(d['ms'], str) and isinstance(d['en'], str):
            manglish.append(d)
    except Exception as e:
        pass
    
data = pd.DataFrame(manglish).to_dict(orient = 'list')
len(data['en'])

986

In [42]:
pairs = [
    ['manglish', 'ms'],
    ['manglish', 'en'],
    ['ms', 'manglish'],
    ['en', 'manglish'],
]

In [44]:
results = {}
for left, right in pairs:
    predicted = []
    l = mapping[right]
    for s in tqdm(data[left]):
        input_ids = tokenizer.encode(f'terjemah ke {l}: {s}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
        outputs = model.generate(input_ids, max_length = 1024)
        predicted.append(tokenizer.batch_decode(outputs, skip_special_tokens = True)[0])
    
    score = chrf.corpus_score(predicted, [data[right][:len(predicted)]])
    results[f'{left}-{right}'] = score

100%|█████████████████████████████████████████| 986/986 [01:35<00:00, 10.28it/s]


In [46]:
results

{'manglish-ms': chrF2++ = 50.76,
 'manglish-en': chrF2++ = 53.16,
 'ms-manglish': chrF2++ = 37.22,
 'en-manglish': chrF2++ = 43.50}

In [47]:
with open('twitter.json_0_to_1000-test.json') as fopen:
    data = json.load(fopen)

twitter = []
for d in data:
    try:
        r = json.loads(d[1])
        d = {
            'pasar ms': d[0],
            'ms': r['malay'],
            'en': r['english'],
        }
        if isinstance(d['pasar ms'], str) and isinstance(d['ms'], str) and isinstance(d['en'], str):
            twitter.append(d)
    except Exception as e:
        pass
    
data = pd.DataFrame(twitter).to_dict(orient = 'list')
len(data['en'])

991

In [48]:
pairs = [
    ['pasar ms', 'ms'],
    ['pasar ms', 'en'],
    ['ms', 'pasar ms'],
    ['en', 'pasar ms'],
]

In [49]:
results = {}
for left, right in pairs:
    predicted = []
    l = mapping[right]
    for s in tqdm(data[left]):
        input_ids = tokenizer.encode(f'terjemah ke {l}: {s}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
        outputs = model.generate(input_ids, max_length = 1024)
        predicted.append(tokenizer.batch_decode(outputs, skip_special_tokens = True)[0])
    
    score = chrf.corpus_score(predicted, [data[right][:len(predicted)]])
    results[f'{left}-{right}'] = score

100%|█████████████████████████████████████████| 991/991 [02:13<00:00,  7.43it/s]


In [50]:
results

{'pasar ms-ms': chrF2++ = 58.74,
 'pasar ms-en': chrF2++ = 54.87,
 'ms-pasar ms': chrF2++ = 53.36,
 'en-pasar ms': chrF2++ = 49.31}